## Notebook 03 – Defining Volatility Regimes

This notebook defines and creates the volatility regimes for the universe (market) as well as the dispersion regimes across tickers, and exports both for downstream analysis.

### Step 0 - Import packages and functions

In [1]:
import sys, os
sys.path.append(os.path.abspath("../src"))

from regime_definitions import compute_market_volatility_regime, compute_cross_sectional_dispersion_regime
from data_loader import load_price_data
import pandas as pd
import numpy as np

C:\Users\edzz0\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


### Step 1 - Load Log Returns dataset

In [2]:
log_returns = pd.read_parquet("../data/processed/log_returns.parquet")

### Step 2 - Compute volatility and dispersion regimes

In [3]:
# Compute volatility regimes using 20-day rolling std of average market returns
regime_vol = compute_market_volatility_regime(log_returns)

# Compute dispersion regimes using 20-day rolling mean of cross-sectional std across tickers
regime_disp = compute_cross_sectional_dispersion_regime(log_returns)

### Step 3 - Export regime datasets

In [4]:
# Save regimes as parquet files
regime_vol.to_frame(name="vol_regime").to_parquet("../data/processed/regime_market_vol.parquet")
regime_disp.to_frame(name="disp_regime").to_parquet("../data/processed/regime_cross_dispersion.parquet")

In [5]:
price = pd.read_parquet("../data/processed/price_filtered.parquet")


In [6]:
old_regime = compute_market_volatility_regime(price, window=21)
new_regime = compute_market_volatility_regime(log_returns, window=21)

In [14]:
np.sum(old_regime != new_regime)

2208

In [18]:
old_regime[old_regime.index == '2025-01-06']

date
2025-01-06    high_vol
dtype: object

In [20]:
(old_regime == new_regime).value_counts()

False    2208
True     1654
dtype: int64